# Initial Setup

In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests
!pip3 install pandas
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip3 install inltk
!pip3 install bnlp_toolkit
!pip3 install wordcloud
!pip3 install git+https://github.com/banglakit/lemmatizer.git#egg=banglakit-lemmatizer
!pip3 install emoji 


!git clone https://github.com/Foysal87/bn_nlp
!git clone https://github.com/banglakit/lemmatizer
# need to include the model for pos tagging manually because github keeps throwing an error whenever i try to do so, the model can be found here - https://github.com/sagorbrur/bnlp/blob/master/model/bn_pos.pkl

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from pprint import pprint
import bn_nlp
import emoji
import regex
import re

In [ ]:
from inltk.inltk import setup
setup('bn')

# Defining the file to be analysed

In [ ]:
reqfile = '/content/filename.txt' # change file name according to whatever you are trying to study and analyse properly

# Counting the number of non-dictionary words, which here are emojis


In [ ]:
emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

In [ ]:
with open(reqfile, 'r') as f:
  lines = f.readlines()

emojis = 0
for line in lines:
  emojis = emojis + len(re.findall(emoj, line))

print(emojis)

# Cleaning the corpora

In [ ]:
def deEmojify(text):
    regex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return regex_pattern.sub(r'',text)

In [ ]:
from re import sub, split, M

with open(reqfile, 'r') as f:
  lines = f.readlines()

cleancorp = ""
linesdone = 0

for line in lines:
    line = sub(r'[\.\,\(\)\-\—\”\“\:\?\n\…\।\‘\_\–\#\¿\?\¡\í\!\\\/\á\[\]\;\&\ó\‘\|\%\'a-zA-Z0-9\—\@\(\)\-\’\=\+\"\>\~\»\*]', '', line) 
    line = sub(r'^[\s\,\.\#\/\'\"\’\:\\\/\"\_]*$', '', line, flags=M)
    line = sub(r'^[\s]*$', '', line, flags=M)
    line = deEmojify(line)
    line = line.strip()
    if len(line) == 0:
      continue
    print(line)
    if len(cleancorp) > 0:
      cleancorp = cleancorp +  "\n" + line
    else:
      cleancorp = line
    linesdone = linesdone + 1

with open('/content/cleancorp.txt', 'w') as ff:
  ff.write(cleancorp)

print("Number of lines: ", linesdone)

# Putting all the individual sentences into a list along with the cleaned corpora sentences 

In [ ]:
with open(reqfile, 'r') as f:
  lines = f.readlines()

sent = []

for line in lines:
  sent.append(line)

with open('/content/cleancorp.txt', 'r') as f:
    lines = f.readlines()

cleansent = []

for line in lines:
    cleansent.append(line)
  
print(len(sent))
print(len(cleansent))

# Count the number of words per sentence

In [ ]:
number = []

for line in cleansent:
  number.append(len(line.split()))

print(number)

# Plottting the distribution of words per sentence 

In [ ]:
from collections import Counter

freq = Counter(number)
print(freq)

In [ ]:
from matplotlib import pyplot as plt

data = np.array(number)
plt.hist(data, bins=np.arange(data.min(), data.max()+1))
plt.show

In [ ]:
dic = {}

for i in range(10, 501, 10):
  dic[i] = 0

for data in freq:
  dic[((data // 10) * 10) + 10] += freq[data]

print(dic)

In [ ]:
from matplotlib import pyplot as plt

x = []
y = []

for thing in dic:
  if dic[thing] != 0:
    x.append(thing)
    y.append(dic[thing])

fig = plt.figure(figsize=(5,2))
plt.xlabel("Values")
plt.ylabel("Frequencies")
plt.title("Plotting the word frequency")
plt.bar(x, y, width=0.5)
plt.plot(x,y)
plt.show()

# Word Tokenization


In [ ]:
from bn_nlp.tokenizer import wordTokenizer
wordtoken=wordTokenizer()

with open('/content/cleancorp.txt', 'r') as f:
    lines = f.readlines()

wordlistbeng = []
linenum = 0
wordcount = 0

for line in lines:
  linenum += 1
  print(linenum)
  tokens = wordtoken.basic_tokenizer(line)
  wordcount = wordcount + len(tokens)
  for word in tokens:
    wordlistbeng.append(word)

print("wordcount: ", wordcount)

### Finding word frequency

In [ ]:
from collections import Counter

wordcountbeng = Counter(wordlistbeng)
print(wordcountbeng.most_common(30))

### Plotting the most frequent words 

In [ ]:
from matplotlib import pyplot as plt
from bn_nlp.preprocessing import ban_processing
bp=ban_processing()

bengalicommonwords = []
bengalicommonwordfreq = []

for thing in wordcountbeng.most_common(30):
  bengalicommonwords.append(bp.bn2enCon(thing[0]))
  bengalicommonwordfreq.append(thing[1])

fig = plt.figure(figsize=(35,6))
plt.xlabel("Words")
plt.ylabel("Frequencies")
plt.title("Plotting the most commonly occuring words")
plt.bar(bengalicommonwords, bengalicommonwordfreq, width=0.5)
plt.plot(bengalicommonwords,bengalicommonwordfreq)
plt.show()

### Removing stop words 

In [ ]:
from bn_nlp.preprocessing import ban_processing
from bn_nlp.tokenizer import wordTokenizer

bp=ban_processing()
wordtoken=wordTokenizer()

nostopbeng = []
wordcountnostop = 0

for line in cleansent:
  wordy = bp.stop_word_remove(line)
  tokens = wordtoken.basic_tokenizer(wordy)
  wordcountnostop = wordcountnostop + len(tokens)
  for word in tokens:
    nostopbeng.append(word)

print("wordcount: ", wordcountnostop)

### Printing frequency of words without the stop words

In [ ]:
from collections import Counter

wordcountbeng = Counter(nostopbeng)
print(wordcountbeng.most_common(30))

### Plotting the most frequent words without the stop words

In [ ]:
from matplotlib import pyplot as plt
from bn_nlp.preprocessing import ban_processing
bp=ban_processing()

bengalicommonwords = []
bengalicommonwordfreq = []

for thing in wordcountbeng.most_common(30):
  bengalicommonwords.append(bp.bn2enCon(thing[0]))
  bengalicommonwordfreq.append(thing[1])

fig = plt.figure(figsize=(35,5))
plt.xlabel("Words")
plt.ylabel("Frequencies")
plt.title("Plotting the most commonly occuring words - without stop words")
plt.bar(bengalicommonwords, bengalicommonwordfreq, width=0.5)
plt.plot(bengalicommonwords,bengalicommonwordfreq)
plt.show()

# POS Tagging


## Data with Stop Words 

In [ ]:
from bnlp import POS
bn_pos = POS()

model_path = "/content/bn_pos.pkl"

bengalitagged = []

for text in cleansent: #this one has stopwords 
  res = bn_pos.tag(model_path, text)

  for thing in res:
    print(thing)
    bengalitagged.append(thing)

### Finding the frequency of each POS tag

In [ ]:
tags = []

for thing in bengalitagged:
  tags.append(thing[1])

tagcountbeng = Counter(tags)

print(tagcountbeng)

### Plotting the 30 most common tags

In [ ]:
from nltk.probability import FreqDist
from matplotlib import pyplot as plt

tags = []
freqs = []

for thing in tagcountbeng.most_common(30):
  tags.append(thing[0])
  freqs.append(thing[1])

fig = plt.figure(figsize=(25,5))
plt.xlabel("Words")
plt.ylabel("Frequencies")
plt.title("Plotting the most commonly occuring POS tags")
plt.bar(tags, freqs, width=0.5)
plt.plot(tags, freqs)
plt.show()

## Data without stop words 

In [ ]:
from bnlp import POS
bn_pos = POS()

model_path = "/content/bn_pos.pkl"

bengalitagged1 = []

res = bn_pos.tag(model_path, nostopbeng)

for thing in res:
  print(thing)
  bengalitagged1.append(thing)

### Finding the frequency of each POS tag

In [ ]:
tags = []

for thing in bengalitagged1:
  tags.append(thing[1])

tagcountbeng = Counter(tags)

print(tagcountbeng)

### Plotting the 30 most common tags 

In [ ]:
from nltk.probability import FreqDist
from matplotlib import pyplot as plt

tags = []
freqs = []

for thing in tagcountbeng.most_common(30):
  tags.append(thing[0])
  freqs.append(thing[1])

fig = plt.figure(figsize=(25,5))
plt.xlabel("Words")
plt.ylabel("Frequencies")
plt.title("Plotting the most commonly occuring POS tags")
plt.bar(tags, freqs, width=0.5)
plt.plot(tags, freqs)
plt.show()

# Stemming 

In [ ]:
from bn_nlp.Stemmer import stemmerOP

stemlist = []

stemmer=stemmerOP()
for word in nostopbeng:
  print(word, "->", stemmer.stem(word))
  stemlist.append(stemmer.stem(word))

### Finding the most common word stems 

In [ ]:
from collections import Counter

stemcountbeng = Counter(stemlist)
print(stemcountbeng.most_common(30))

### Plotting the most common stems 

In [ ]:
from matplotlib import pyplot as plt
from bn_nlp.preprocessing import ban_processing
bp=ban_processing()

commonstems = []
commonstemfreq = []

for thing in stemcountbeng.most_common(30):
  commonstems.append(bp.bn2enCon(thing[0]))
  commonstemfreq.append(thing[1])

fig = plt.figure(figsize=(35,5))
plt.xlabel("Words")
plt.ylabel("Frequencies")
plt.title("Plotting the most commonly occuring stems")
plt.bar(commonstems, commonstemfreq, width=0.5)
plt.plot(commonstems,commonstemfreq)
plt.show()

# Word Cloud 

In [ ]:
from collections import Counter
from bn_nlp.posTag import postag

tagger=postag()
wordcountbeng = Counter(nostopbeng)

pos = []

length = len(wordcountbeng)

for thing in wordcountbeng.most_common(2000):
  possy = tagger.tag(thing[0])
  for possed in possy:
    pos.append(possed[1])

bengcommonwords = []
bengcommonwordfreq = []

index = 0
i = 0

for thing in wordcountbeng.most_common(2000):
  if 'noun' == pos[index]:
    bengcommonwords.append(thing[0])
    bengcommonwordfreq.append(thing[1] * 5)
    i += 1
  elif 'adjective' in pos[index]:
    bengcommonwords.append(thing[0])
    bengcommonwordfreq.append(thing[1] * 3)
    i += 1
  elif 'adverb' in pos[index]:
    bengcommonwords.append(thing[0])
    bengcommonwordfreq.append(thing[1] * 2)
    i += 1
  if 'verb' == pos[index]:
    bengcommonwords.append(thing[0])
    bengcommonwordfreq.append(thing[1] * 2.5)
    i += 1
  
  if i == 50:
    break
  index += 1

In [ ]:
n = len(bengcommonwords)

for i in range(n-1):
  for j in range(0, n-i-1):
    if bengcommonwordfreq[j] < bengcommonwordfreq[j + 1] :
            bengcommonwordfreq[j], bengcommonwordfreq[j + 1] = bengcommonwordfreq[j + 1], bengcommonwordfreq[j]
            bengcommonwords[j], bengcommonwords[j + 1] = bengcommonwords[j + 1], bengcommonwords[j]

In [ ]:
from matplotlib import pyplot as plt
from bn_nlp.preprocessing import ban_processing
bp=ban_processing()

bengplot = []

for word in bengcommonwords:
  bengplot.append(bp.bn2enCon(word))

fig = plt.figure(figsize=(40,5))
plt.xlabel("Words")
plt.ylabel("Frequencies")
plt.title("Plotting the word cloud words")
plt.bar(bengplot, bengcommonwordfreq, width=0.5)
plt.plot(bengplot, bengcommonwordfreq)
plt.show()

In [ ]:
for i in range(len(bengcommonwords)):
  print(bengcommonwords[i])

# Word Cloud Visualization

In [ ]:
from bn_nlp.preprocessing import ban_processing
bp=ban_processing()

wcdict = {}

for i in range(len(bengcommonwords)):
  wcdict[bp.bn2enCon(bengcommonwords[i])] = bengcommonwordfreq[i]

print(wcdict)

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(background_color='black', width = 500, height=500, margin=2)

In [ ]:
wc.fit_words(wcdict)
wc.to_file('wc.png')

In [ ]:
wcdict = {}

for i in range(len(bengcommonwords)):
  wordy = bengcommonwords[i]
  wcdict[wordy] = bengcommonwordfreq[i]

print(wcdict)